In [3]:
import os
import sys

from tqdm import tqdm

from Bio.Seq import Seq
import gzip
from twobitreader import TwoBitFile
import pandas as pd
from collections import defaultdict
import numpy as np

### Load Acetylomics Input

In [6]:
INPUT = './inputs/acetylomics/brca_acetyl_binary.csv'

In [7]:
df = pd.read_csv(INPUT, index_col=0)
df.head()

,X11BR047,X11BR043,X11BR049,X11BR023,X18BR010,X06BR003,X11BR074,X18BR017,X01BR017,X06BR006,...,X09BR001,X03BR011,X11BR036.REP2,X01BR010,invented_patient,accession_number,geneSymbol,variableSites,accession_numbers,site_position
NP_000468.1_K28k _1_1_28_28,1,0,0,1,1,1,0,0,0,1,...,1,1,1,1,1,NP_000468.1,ALB,K28k,NP_000468.1,28
NP_000468.1_K36k _1_1_36_36,1,0,0,0,1,1,0,0,0,1,...,1,1,1,1,1,NP_000468.1,ALB,K36k,NP_000468.1,36
NP_000468.1_K44k _1_1_44_44,1,0,1,1,1,1,1,0,0,1,...,1,1,1,1,1,NP_000468.1,ALB,K44k,NP_000468.1,44
NP_000468.1_K65k _1_1_65_65,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,NP_000468.1,ALB,K65k,NP_000468.1,65
NP_000468.1_K75k _1_1_75_75,1,0,0,1,1,1,0,1,0,0,...,1,1,1,1,1,NP_000468.1,ALB,K75k,NP_000468.1,75


### Add Uniprot IDs

In [4]:
map_df = pd.read_csv('./acetylomics_inputs/accession_to_uniprot.tsv',sep='\t')
mapping = map_df.set_index('accession_number').to_dict()['uniprot_id']

def mapper(x):
    try:
        return mapping[x]
    except:
        return None

df['uniprot_id'] = df['accession_number'].apply(lambda x: mapper(x))
df = df.dropna()
df.shape

(10967, 160)

In [5]:
df[df['geneSymbol']=='TP53']

,X11BR047,X11BR043,X11BR049,X11BR023,X18BR010,X06BR003,X11BR074,X18BR017,X01BR017,X06BR006,...,X03BR011,X11BR036.REP2,X01BR010,invented_patient,accession_number,geneSymbol,variableSites,accession_numbers,site_position,uniprot_id
NP_000537.3_K120k _1_1_120_120,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,NP_000537.3,TP53,K120k,NP_000537.3,120,P04637
NP_000537.3_K132k _1_1_132_132,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,NP_000537.3,TP53,K132k,NP_000537.3|NP_001119590.1,132,P04637
NP_000537.3_K305k _1_1_305_305,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,NP_000537.3,TP53,K305k,NP_000537.3|NP_001119590.1|NP_001119585.1|NP_0...,305,P04637
NP_000537.3_K319k _1_1_319_319,1,1,1,1,1,1,1,1,1,0,...,0,1,0,1,NP_000537.3,TP53,K319k,NP_000537.3|NP_001119586.1,319,P04637
NP_000537.3_K357k _1_1_357_357,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,NP_000537.3,TP53,K357k,NP_000537.3,357,P04637


### Fitler by Patients and Melt Dataframe

In [45]:
pats = [x for x in list(df) if x.startswith('X')] + ['RetroIR','CPT001846','CPT000814','RetroIR.REP1','invented_patient']
input_df = df.melt(id_vars=[m for m in list(df) if m not in pats]).set_index(['variable','uniprot_id','site_position']).reset_index().rename(columns={'variable':'patient'})

In [46]:
### Output Files
freq_file = 'acetyl_freqs.txt'
split_protein_dir = 'acetyl_protein_dir'

# Make frequency file in CLUMPS Format
freq_df = input_df.groupby('patient').sum().sort_values('value').rename(columns={'value':'raw'}).drop(columns='site_position')
freq_df['log10'] = np.log10(freq_df['raw'])

meanmf = np.mean(freq_df['log10'])
sdmf = np.std(freq_df['log10'])
freq_df['zlog'] = 1.0 / (np.log10(freq_df['raw'] - meanmf) / sdmf)
freq_df['rank_score'] = freq_df.reset_index().reset_index()['index'].astype(float).values / float(freq_df.shape[0])

# Rename for formatting
freq_df = freq_df.reset_index().rename(columns={'patient':'SAMPLE','raw':'MUT_COUNT', 'rank_score':'TTYPE_RANK_SCORE','zlog':'ZLOG_SCORE'})
freq_df['TTYPE'] = 'BRCA'
freq_df.loc[:,['TTYPE','SAMPLE','MUT_COUNT','TTYPE_RANK_SCORE','ZLOG_SCORE']].to_csv(freq_file,sep='\t',index=None)

### Format CLUMPS Input "Muts" file

* `A`: acetylation event

In [47]:
# Make muts file in CLUMPS Format
muts_df = input_df[input_df['value'] == 1].loc[:,['patient','uniprot_id','site_position','geneSymbol', 'variableSites']]
muts_df['ttype'] = 'BRCA'
muts_df['fill'] = 'na'
muts_df['mtype'] = 'A'

muts_df = muts_df.loc[:,['ttype','patient','fill','uniprot_id','variableSites','site_position','mtype']]

In [48]:
# Create split protein Directory
split_protein_dir = 'acetyl_protein_dir'

if not os.path.exists(split_protein_dir):
    print("{} exists.".format(split_protein_dir))
    os.makedirs(split_protein_dir)

for uniprot in tqdm(set(muts_df['uniprot_id'])):
    if uniprot is not np.nan:
        muts_df[muts_df['uniprot_id']==uniprot].to_csv(os.path.join(split_protein_dir,uniprot), sep='\t',header=None, index=None)

100%|██████████| 2322/2322 [00:52<00:00, 43.95it/s]
